## Set Covering

In the first part, I try to analyze professor's method for optimising this problem. After analysis I try to optimize the code and propose new methods that can be used.

In [12]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [13]:
State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

Three approaches have been used to address this problem:

### 1- Depth First
This algorithm starts at the root (top) node of a tree and goes as far as it can down a given branch (path), then backtracks until it finds an unexplored path, and then explores it.
### 2- Breadth First
It starts at the tree root and explores all nodes at the present depth prior to moving on to the nodes at the next depth level. Extra memory, usually a queue, is needed to keep track of the child nodes that were encountered but not yet explored.
### 3- Greedy Best First
The algorithm works by using a heuristic function to determine which path is the most promising. The heuristic function takes into account the cost of the current path and the estimated cost of the remaining paths. If the cost of the current path is lower than the estimated cost of the remaining paths, then the current path is chosen. This process is repeated until the goal is reached.

In [14]:
PROBLEM_SIZE = 20
NUM_SETS = 40
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

## Depth First

In [15]:
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.pop()

with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.append(new_state)
        current_state = frontier.pop()


print(f"Depth first solved in {counter:,} steps ({len(current_state.taken)} tiles)")

0it [00:00, ?it/s]

Depth first solved in 13 steps (13 tiles)


## Breadth First

In [5]:
'''
Due to the high computational cost I was not able to calculate the Breadth First search.
'''



frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.popleft()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.append(new_state)
        current_state = frontier.popleft()
        pbar.update(1)

print(f"Breadth First solved in {counter:,} steps ({len(current_state.taken)} tiles)")






0it [00:00, ?it/s]

KeyboardInterrupt: 

## Greedy Best First

In [16]:
def f(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size

In [17]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Greedy Best First solved in {counter:,} steps ({len(current_state.taken)} tiles)")

0it [00:00, ?it/s]

Greedy Best First solved in 5 steps (5 tiles)


## A*

In [18]:
def h_by_professor(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def f_by_professor(state):
    return len(state.taken) + h_by_professor(state)

In [19]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f_by_professor(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f_by_professor(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)
        
        
print(f"A_star objective by professor solved in {counter:,} steps ({len(current_state.taken)} tiles)")

0it [00:00, ?it/s]

A_star objective by professor solved in 443 steps (5 tiles)


single_state() function just considers covered status of one single selected set.

In [20]:
def single_state(number, num_sets):
    state = State(set({number}), set(range(num_sets)))
    state.not_taken.remove(number)
    return state

### LAB01 Assignment
I tried to create my own optimum A* to minimize the tiles. In this function I minimize number of True labels existed in current covered indexes existed in not taken states. I also tried to minimize number of False labels in current not covered indexes exists in not taken states.

We have good results and in some cases number of tiles are lower than the first greedy best approach. It also outperforms the proposed cost function by professor.

In [21]:
def h_proposed(state):
    coverd_problems = covered(state)
    not_covered_indexes = np.where(np.logical_not(coverd_problems))[0]
    covered_indexes = np.where(coverd_problems)[0]
    not_taken_states = state.not_taken
    missing_size = PROBLEM_SIZE - sum(covered(state))
    
    tot_false_not_covered = 0
    tot_truth_covered = 0
    for ind_state in not_taken_states:
        tot_false_not_covered += sum(np.logical_not([covered(single_state(ind_state, NUM_SETS))[n] for n in not_covered_indexes]))
        tot_truth_covered += sum([covered(single_state(ind_state, NUM_SETS))[n] for n in covered_indexes])
        
    return len(state.taken) + tot_truth_covered + tot_false_not_covered

In [22]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((h_proposed(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((h_proposed(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"A_star objective by me solved in {counter:,} steps ({len(current_state.taken)} tiles)")

0it [00:00, ?it/s]

A_star objective by me solved in 5 steps (5 tiles)


### Reviewing other students work
In this section I try to test other students A* and compare the results with myself. 

In [25]:
'''
Ferrigno - s316467
'''
def h(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_elements = PROBLEM_SIZE - np.sum(already_covered)
    average_set_size = np.mean([np.sum(s) for s in SETS])
    return missing_elements / average_set_size

def fProf(state):
    return len(state.taken) + hProf(state)

def f(state):
    return len(state.taken) + h(state)





frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

0it [00:00, ?it/s]

Solved in 443 steps (5 tiles)


In [30]:
'''
Tilocca - s305938
'''


def array_with_more_true(arrays, missing):
    num_true_per_array = [np.count_nonzero(array) for array in arrays]#create a vector with number of cell true 
    array_with_count = list(zip(arrays, num_true_per_array))#create an array with sets, n_true
    array_with_count.sort(key=lambda x: x[1], reverse=True)#reorder
    element_in_arrays = [x[1] for x in array_with_count]
    cnt=0#cnt si incrementa fino a che non copro tutte le missing size
    cov=missing
    for i in element_in_arrays:
       if missing>0:
           cnt+=1
           missing=missing-i
    top_n_arrays = [x[0] for x in array_with_count[:cnt]]

    return top_n_arrays,cnt

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def h(state):
    
    missing_size = PROBLEM_SIZE - sum(covered(state))
    x=tuple(SETS[i] for i in state.not_taken)
    _,optimistic_estimate=array_with_more_true(x,missing_size)
    return optimistic_estimate




#we need a queue that is the frontier
frontier=PriorityQueue()
#define the initial state
frontier.put(State(set(),set(range(NUM_SETS))))
current_state=frontier.get()
contatore=0
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        for action in current_state[1]:
         contatore+=1
         new_state=State (current_state.taken ^ {action}, current_state.not_taken ^{action})

         frontier.put((((h(new_state)-len(new_state.taken))),new_state))
        _,current_state=frontier.get()
        pbar.update(1)

    
print(f"Solved in {contatore:,} steps ({len(current_state.taken)} tiles)")


0it [00:00, ?it/s]

Solved in 520 steps (16 tiles)


### Summary

    1- Depth First:                                  13  steps (13 tiles)
    2- Greedy Best First:                            5   steps (5 tiles)
    3- Professor Method (admissible heuristic):      443 steps (5 tiles)
    4- My Method:                                    5   steps (5 tiles)
    